This notebook empirically confirms Theorem 1, which states that the iterates of the joint NMF and PLSA algorithms are identical up to a trivial scaling of the topic weights.

We run both algorithms with the same initialization and for a fixed number of iterations, then compare the resulting iterates. This process is repeated across multiple random initializations and different number of iterations.

**Note:**

Although Algorithms 1, S2, and S3 have iterates that are identical up to a trivial constant, they converge in a different number of iterations when the stopping criterion is the relative change in objective function value. This is expected because the loss function values of the iterates differ by a data-dependent constant $C=C(X)$, assuming identical initializations.

In [1]:
import numpy as np

import klnmf

In [2]:
X = np.load("../data/olivetti/faces.npy")
per_sample_sum = np.sum(X, axis=0)
N_TOPICS = 6
N_INITS = 5
MAX_N_ITER = 100

In [3]:
for n in range(N_INITS):
    n_iter = np.random.choice(range(1, MAX_N_ITER))
    model_nmf = klnmf.KLNMF(
        n_topics=N_TOPICS,
        update_method="mu-jointnorm1",
        min_iterations=n_iter,
        max_iterations=n_iter,
    )
    model_nmf.fit(X, seed=n)
    model_plsa = klnmf.KLNMF(
        n_topics=N_TOPICS,
        update_method="mu-jointnorm2",  # PLSA
        min_iterations=n_iter,
        max_iterations=n_iter,
    )
    model_plsa.fit(X, seed=n)
    assert np.allclose(model_nmf.W, model_plsa.W)
    for k in range(N_TOPICS):
        assert np.allclose(model_nmf.H[k, :] / model_plsa.H[k, :], per_sample_sum)

print("done")

done
